In [1]:
import pandas as pd
import numpy as np
import vcf
import h5py
import csv
import os
from random import seed, random

# Generate GWAS Data for Semantic Zoom

We will use a GWAS *.tsv file (https://www.ebi.ac.uk/gwas/docs/file-downloads) as an input to generate two following files:
1. HiGlass' BEDDB file: This file selectively stores variant information for given zoom level.
2. HiGlass' multivec file: This file stores the frequency of variant information for every binned genomic regions.

These two files can be used together in a single Gosling.js track to allow semantic zooming:
- When zoomed out, show density information
- When zoomed in, show detailed information


clodius aggregate bedfile --chromsizes-filename hg38.txt --delimiter $'\t' --importance-column 6 --max-per-tile 80 gwas.v1.bed 

python manage.py ingest_tileset --filename data/gwas.v1.beddb --filetype beddb --datatype bedlike --uid gwas-v1-beddb

## Create The Two Files

In [30]:
original_file = os.path.join('data', 'gwas_catalog_v1.0-associations_e100_r2021-03-25.tsv')
bed_file = os.path.join('data', 'gwas.bed')
mv5_file = os.path.join('data', 'gwas.hdf5')
chr_file = os.path.join('data', 'hg38_full.txt')

In [31]:
# We are interested only in the following information
CHR_FILTER = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

In [32]:
df = pd.read_csv(chr_file, '\t', header=None, names=['chr', 'length'])

df = df[df.chr.isin(['chr' + c for c in CHR_FILTER])]

TOTAL_CHROM_LEN = df.length.sum()

# l = df[df.chr == 'chr1'].length
# l

In [33]:
odf = pd.read_csv(original_file, '\t')
odf.head()

/Users/slyi/anaconda3/envs/gos/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (9,11,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,CONTEXT,INTERGENIC,RISK ALLELE FREQUENCY,P-VALUE,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV
0,2020-06-04,32296059,Han Y,2020-04-15,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/32296059,Genome-wide analysis highlights contribution o...,Asthma,"84,492 European ancestry cases, 2,149 African ...",NR,...,synonymous_variant,0.0,0.57,2.000000e-09,8.698970,NaN,NaN,NaN,NR [> 9500000] (imputed),N
1,2020-02-05,31666681,Zhao B,2019-10-30,Mol Psychiatry,www.ncbi.nlm.nih.gov/pubmed/31666681,Large-scale GWAS reveals genetic architecture ...,White matter microstructure (axial diusivities),"17,706 European ancestry individuals",NaN,...,intron_variant,0.0,NaN,3.000000e-09,8.522879,(Inferior fronto-occipital fasciculus),NaN,NaN,NR [8955960] (imputed),N
2,2020-02-05,31666681,Zhao B,2019-10-30,Mol Psychiatry,www.ncbi.nlm.nih.gov/pubmed/31666681,Large-scale GWAS reveals genetic architecture ...,White matter microstructure (axial diusivities),"17,706 European ancestry individuals",NaN,...,intron_variant,0.0,NaN,2.000000e-08,7.698970,(Inferior fronto-occipital fasciculus),NaN,NaN,NR [8955960] (imputed),N
3,2020-06-04,32296059,Han Y,2020-04-15,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/32296059,Genome-wide analysis highlights contribution o...,Asthma,"84,492 European ancestry cases, 2,149 African ...",NR,...,intron_variant,0.0,0.12,8.000000e-09,8.096910,NaN,NaN,NaN,NR [> 9500000] (imputed),N
4,2020-02-05,31666681,Zhao B,2019-10-30,Mol Psychiatry,www.ncbi.nlm.nih.gov/pubmed/31666681,Large-scale GWAS reveals genetic architecture ...,White matter microstructure (axial diusivities),"17,706 European ancestry individuals",NaN,...,intron_variant,0.0,NaN,3.000000e-09,8.522879,(Inferior fronto-occipital fasciculus),NaN,NaN,NR [8955960] (imputed),N


In [67]:
CATEGORY_FILTER = [
    'Anthracycline-induced cardiotoxicity in early breast cancer',
    'Invasive breast cancer',
    'Persistent chemotherapy-induced alopecia in breast cancer',
    'Sensory peripheral neuropathy in microtubule targeting agent-treated breast cancer',
    'Adverse response to chemotherapy (amenorrhea) in breast cancer',
    'Acute post-radiotherapy pain in breast cancer',
    'BRCA1/2-negative high-risk breast cancer',
    'BRCA1 mutation in breast cancer',
    'BRCA2 mutation in breast cancer',
    'Survival in breast cancer (estrogen-receptor positive)',
    'Plasma anastrozole concentration in anastrozole-treated estrogen receptor positive breast cancer',
    'Breast cancer specific mortality in breast cancer',
    'Breast cancer specific mortality in estrogen receptor negative breast cancer',
    'Breast cancer specific mortality in estrogen receptor positive breast cancer',
    'Response to tamoxifen in oestrogen receptor positive/HER2 negative breast cancer',
    'Disease-free survival in breast cancer',
    'Estrone conjugates/estrone ratio in resected early stage estrogen-receptor positive breast cancer',
    'Estrone/androstenedione ratio in resected early stage-receptor positive breast cancer',
    'Plasma estrone levels in resected estrogen-receptor positive breast cancer',
    'Plasma androstenedione levels in resected early stage-receptor positive breast cancer',
    'Plasma estrone conjugates levels in resected early stage estrogen-receptor positive breast cancer',
    'Lobular breast cancer (menopausal hormone therapy interaction)',
    'Estrogen receptor status in HER2 negative breast cancer',
    'Estrogen receptor status in breast cancer',
    'Taxane-induced peripheral neuropathy in breast cancer',
    'HER2 status in breast cancer',
    'Survival in endocrine treated breast cancer (estrogen-receptor positive)',
    'Response to tamoxifen in breast cancer',
    'Estradiol plasma levels (breast cancer)',
    'Adverse response to chemotherapy in breast cancer (alopecia) (anti-microtubule)',
    'Adverse response to chemotherapy in breast cancer (alopecia)',
    'Adverse response to chemotherapy in breast cancer (alopecia) (cyclophosphamide+epirubicin+/-5FU)',
    'Adverse response to chemotherapy in breast cancer (alopecia) (cyclophosphamide+doxorubicin+/-5FU)',
    'Adverse response to chemotherapy in breast cancer (alopecia) (paclitaxel)',
    'Adverse response to chemotherapy in breast cancer (alopecia) (docetaxel)',
    'Response to chemotherapy in breast cancer (hypertension) (bevacizumab)',
    'Response to chemotherapy in breast cancer hypertensive cases (cumulative dose) (bevacizumab)',
    'Change in LVEF in response to paclitaxel and trastuzumab in HER2+ breast cancer'
]

diseases = odf[odf['DISEASE/TRAIT'].str.contains("breast cancer", na=False)]['DISEASE/TRAIT'].unique().tolist()

len(diseases)
len(odf['DISEASE/TRAIT'].unique().tolist())

5052

In [68]:
# For the extracted keys, refer to https://www.ebi.ac.uk/gwas/docs/fileheaders
# DATE ADDED TO CATALOG	PUBMEDID	FIRST AUTHOR	DATE	JOURNAL	LINK	STUDY	DISEASE/TRAIT	INITIAL SAMPLE SIZE	REPLICATION SAMPLE SIZE	REGION	CHR_ID	CHR_POS	REPORTED GENE(S)	MAPPED_GENE	UPSTREAM_GENE_ID	DOWNSTREAM_GENE_ID	SNP_GENE_IDS	UPSTREAM_GENE_DISTANCE	DOWNSTREAM_GENE_DISTANCE	STRONGEST SNP-RISK ALLELE	SNPS	MERGED	SNP_ID_CURRENT	CONTEXT	INTERGENIC	RISK ALLELE FREQUENCY	P-VALUE	PVALUE_MLOG	P-VALUE (TEXT)	OR or BETA	95% CI (TEXT)	PLATFORM [SNPS PASSING QC]	CNV

mv5_data = h5py.File(mv5_file, "w")

seed(1)

output = []
test = []
density_dict = { c : np.zeros((df[df.chr == ('chr' + c)].length.sum(), 1)) for c in CHR_FILTER}

for index, record in odf.iterrows(): # Oh, this is so wrong..
    # print(record)

    # if not record['DISEASE/TRAIT'] in CATEGORY_FILTER:
        # continue

    if not record.CHR_ID in CHR_FILTER:
        continue
    
    if not str(record.CHR_POS).isnumeric():
        continue
        
    #print(record)
    
    """
    BED
    """
    data_clean = {
        'chr': 'chr' + record.CHR_ID, 
        'start': int(record.CHR_POS),
        'end': int(record.CHR_POS) + 1,
        'pubmedid': record.PUBMEDID,
        'date': record.DATE,
        'link': record.LINK,
        'pval': record['P-VALUE'],
        'importance': 1 / (record['P-VALUE'] + 0.0001) + random(),
        'disease': record['DISEASE/TRAIT'],
        'pvalmlog': record['PVALUE_MLOG'],
        'pvaltext': record['P-VALUE (TEXT)'],
    }
    
    output.append(data_clean)
    
    """
    MV5
    """    
    column_index = 0
    
    # Add frequency by 1
    # print(record.CHR_ID, record.CHR_POS, column_index)
    density_dict[record.CHR_ID][int(record.CHR_POS), column_index] = density_dict[record.CHR_ID][int(record.CHR_POS), column_index] + 1

print("Writing to BED file")
headers = output[0].keys()
with open(bed_file, 'w') as opf:
    myWriter = csv.DictWriter(opf, delimiter='\t', fieldnames=headers)
    for row in output:
        myWriter.writerow(row)

print("Writing to hdf5 file")
for c in CHR_FILTER:
    print("chr" + c)
    mv5_data.create_dataset(('chr' + c), (df[df.chr == ('chr' + c)].length.sum(), 1), "i", data=density_dict[c], compression='gzip')

mv5_data.close()

Writing to BED file
Writing to hdf5 file
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY


Then, run the following commands:

```sh
clodius aggregate bedfile \
    --chromsizes-filename hg38_full.txt \
    --delimiter $'\t' \
    --importance-column 8 \
    --max-per-tile 160 \
    gwas.bed

clodius aggregate multivec \
    --chromsizes-filename hg38_full.txt \
    --starting-resolution 1 \
    gwas.hdf5

python manage.py ingest_tileset \
    --filename gwas.bed.beddb \
    --filetype beddb \
    --datatype bedlike \
    --uid gwas-beddb
```